--Part 1--

In [56]:
# valve open at time=t will release flowrate*(30-t)
# destination => open the valve  # no need to go back as destination 
# current_loc => destination : distance = d; t-=d; acc+=flowrate*(30-t) 
# go through all possible paths: n!
# measure distance from any valve to any other valve => floyd warshall
# Valve["AA"].adj => {"DD":1, "II":1,"BB":1}

import re
import math, copy 

def parse_string(string):
    #split ether "Valve " or " has flow rate=" or "; tunnels lead to valves " or "tunnel leads to valve"
    match = re.search(r'tunnels lead to valves', string)
    if match:
        string = re.split("Valve | has flow rate=|; tunnels lead to valves ", string)
    else:
        string = re.split("Valve | has flow rate=|; tunnel leads to valve ", string)
    string = [i for i in string if i]
    valve=string[0]
    flow_rate=int(string[1])
    adj=string[2].split(", ")
    return valve, flow_rate, adj 

# print(parse_string("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB"))


class Valve:
    def __init__(self, name, flow_rate, adj:dict):
        self.name = name
        self.flow_rate = flow_rate
        self.adj = adj

with open('/Users/yang/Desktop/adventofcode/d16_valve.txt') as file:
    lines=file.read().splitlines()

valves = {}
#add the starting point even if it is 0 flow_rate
#not working because then some are not connected

       
valves = {}
#can also make valves a list
for line in lines:
    valve, flow_rate, adj = parse_string(line)
    #make adj a dict with value as 1
    adj = {i:1 for i in adj}
    valves[valve] = Valve(valve, flow_rate, adj)
 

# fill in the distance between any two valves for floyd warshall
for i in valves.keys():
    for j in valves.keys():
        if i not in valves[j].adj:
            valves[j].adj[i] = 999

# print(valves["BB"].adj)
# print(valves["BB"].adj["HH"])

#use floyd warshall to find shortest path between any two valves
for k in valves.keys():
    for i in valves.keys():
        for j in valves.keys():
            valves[j].adj[i]=min(valves[j].adj[i], valves[j].adj[k]+valves[k].adj[i])
            valves[i].adj[j]=min(valves[j].adj[i], valves[j].adj[k]+valves[k].adj[i])

# get rid of dict items with 0 flow_rate
# input starting vertex
start="AA"

#optimize the graph
valves_copy = valves.copy()
for i in valves_copy.keys():
    if valves[i].flow_rate == 0 and i != start:
        valves.pop(i)

valves_copy = copy.deepcopy(valves)
for i in valves_copy.keys():
    for j in valves_copy[i].adj.keys():
        if j not in valves.keys():
            valves[i].adj.pop(j)

# for i in valves.keys():
#     print(i, valves[i].flow_rate, valves[i].adj)

def find_all_paths2(graph, start, time, path=[], acc=0, acc_list=[], path_list=[]):
    path.append(start)
    acc+=graph[start].flow_rate*(time)
    if len(path)==len(graph):
        # print(path)
        path_list.append(path)
        acc_list.append(acc)
    else:
        i=0
        for k in graph.keys():
            time_copy=time
            if k not in path:
                # if path == ['AA', 'DD', 'BB', 'JJ', 'HH']:
                #     print(k)
                distance = valves[start].adj[k]
                # print(start,k,distance)
                if time_copy-distance>0:
                    # print("time:",time,"distance:",distance,"k",k, "flowrate", valves[k].flow_rate, "acc:",acc)
                    i+=1
                    path_copy = path.copy() #need to copy the path
                    time_copy-=distance
                    time_copy-=1
                    find_all_paths2(graph, k, time_copy, path_copy, acc, acc_list, path_list)
                else:
                    path_list.append(path)### add to path_list even if it is not a complete path
                    acc_list.append(acc)###
            else:
                continue
    return acc_list, path_list
"""
acc_list = []
path_list = []
# find all possible paths and calculate the total flow rate
# time is the time left
# some valves may not be able to open when time is 0, but still need to add to path
def find_all_paths(graph, start, time, path=[], acc=0):
    path.append(start)
    acc+=graph[start].flow_rate*(time)
    if len(path)==len(graph):
        # print(path)
        path_list.append(path)
        acc_list.append(acc)
    else:
        i=0
        for k in graph.keys():
            time_copy=time
            if k not in path:
                # if path == ['AA', 'DD', 'BB', 'JJ', 'HH']:
                #     print(k)
                distance = valves[start].adj[k]
                # print(start,k,distance)
                if time_copy-distance>0:
                    # print("time:",time,"distance:",distance,"k",k, "flowrate", valves[k].flow_rate, "acc:",acc)
                    i+=1
                    path_copy = path.copy() #need to copy the path
                    time_copy-=distance
                    time_copy-=1
                    find_all_paths(graph, k, time_copy, path_copy, acc)
                else:
                    path_list.append(path)### add to path_list even if it is not a complete path
                    acc_list.append(acc)###
            else:
                continue

find_all_paths2(valves, start, 30)
"""

acc_list, path_list = find_all_paths2(valves, start, 30)

print(max(acc_list))
# print(path_list[0])
# print(path_list[-1])
print(path_list[acc_list.index(max(acc_list))])

print(len(path_list))
print(len(acc_list))
# for i in path_list:
#     print(i)

#AA-DD-BB-JJ-HH-EE-CC
            

1376
['AA', 'HS', 'EG', 'TJ', 'HO', 'IQ']
1988582
1988582


--Part 2--

In [58]:
# divide destinations into two groups
# each group to find a way to release the most
# find all possible combinations of two groups

# find all possible paths and calculate the total flow rate
# time is the time left
# some valves may not be able to open when time is 0, but still need to add to path
def find_all_paths3(graph, keys, start, time, path=[], acc=0, acc_list=[], path_list=[]):
    # print("ini",path)
    path.append(start)
    acc+=graph[start].flow_rate*(time)
    if len(path)==len(keys):
        # print(path)
        path_list.append(path)
        acc_list.append(acc)
    else:
        i=0
        for k in keys:  #if start is not in keys, then the last key will not be in loop
            time_copy=time
            if k not in path:
                # if path == ['AA', 'DD', 'BB', 'JJ', 'HH']:
                #     print(k)
                distance = graph[start].adj[k]
                # print(start,k,distance)
                if time_copy-distance>0:
                    # print("time:",time,"distance:",distance,"k",k, "flowrate", valves[k].flow_rate, "acc:",acc)
                    i+=1
                    path_copy = path.copy() #need to copy the path
                    time_copy-=distance
                    time_copy-=1
                    # acc_list_copy = acc_list.copy()
                    # path_list_copy = path_list.copy()
                    find_all_paths3(graph, keys, k, time_copy, path_copy, acc, acc_list, path_list)
                else:
                    path_list.append(path)### add to path_list even if it is not a complete path
                    acc_list.append(acc)###
            else:
                continue
    return acc_list, path_list

#necessary to initialize empty list
# acc_l, path_l = find_all_paths3(valves, ['AA', 'DD', 'BB'], start, 26, path=[], acc=0, acc_list=[], path_list=[])
# print("test",acc_l[-1:], path_l[-1:],"\n")
# print(acc_l)
# print("path_l",path_l)
# acc_l, path_l = find_all_paths3(valves, ['AA', 'BB','HH'], start, 26, path=[],acc=0)
# print("test",acc_l[-3:], path_l[-3:])
# print(acc_l)
# print("path_l",path_l)
# acc_l, path_l = find_all_paths3(valves, ['AA', 'DD','HH','JJ'], start, 26, path=[])
# print("test",acc_l[-3:], path_l[-3:])
# print(acc_l)
# print("path_l",path_l)


In [59]:

def find_all_combinations(graph):
    #find all possible combinations of two groups
    #each group has at least one valve
    #each group has at least one valve with flow_rate>0
    #each group has at least one valve with flow_rate>0 and is not the starting point
    group1_list = []
    group2_list = []
    k = 0
    for i in range(2**(len(graph)-1)):
        group1 = []
        group2 = []
        group1.append(start)
        group2.append(start)
        keys=list(graph.keys())
        keys.remove(start)
        j = 0
        for key in keys:
            if (i >> j) & 1:  #i >> j means i/2^j
                group1.append(key)
            else:
                group2.append(key)
            j+=1
        if len(group1)>1 and len(group2)>1:
            k+=1
            # if k//10 == 0:
            #     print(group1, group2)
            group1_list.append(group1.copy())
            group2_list.append(group2.copy())
            # print(group1_list, group2_list)
    return group1_list, group2_list

group1,group2 = find_all_combinations(valves)
print("group1_list:",len(group1))
print("group2_list:",len(group2))
print("group2_0",group1[:3])
print("group2_0",group2[:3])
print(2**(len(valves)-1))


#test
# print("group1[3]",group1[3])
# acc_l, path_l = find_all_paths3(valves, group1[3], start, 26, path=[], acc=0, acc_list=[], path_list=[])
# print("test1",acc_l[:3], path_l[:3])
# print("group2[3]",group2[3])
# acc_l, path_l = find_all_paths3(valves, group2[3], start, 26, path=[], acc=0, acc_list=[], path_list=[])
# print(path_l)
# print("test2",acc_l[:3], path_l[:3])

# loop through all combinations of two groups
# find the best combination
# find the best path for each group


#group1: [[BB],          [],...]
#group2: [[CC,DD,EE,...],[],...]

#test
# acc_l1, path_l1 = find_all_paths3(valves, ['AA','BB', 'CC', 'JJ'], start, 26, path=[], acc=0, acc_list=[], path_list=[])
# acc_l2, path_l2 = find_all_paths3(valves, ['AA','DD', 'EE', 'HH'], start, 26, path=[], acc=0, acc_list=[], path_list=[])
# print(max(acc_l1), path_l1[acc_l1.index(max(acc_l1))])
# print(max(acc_l2), path_l2[acc_l2.index(max(acc_l2))])


acc_list = []
path_list_final= []
for i in range(len(group1)):
    #find all possible paths for group1[i], group2[i]
    #=> opt_path1[i]: acc1[i], opt_path2[i]:acc2[i]  ==>  acc1[i]+acc2[i]
    # print(i, "group1[i],group2[i]",group1[i], group2[i])

    acc_l1, path_l1 = find_all_paths3(valves, group1[i], start, 26, path=[], acc=0, acc_list=[], path_list=[])
    acc_l2, path_l2 = find_all_paths3(valves, group2[i], start, 26, path=[], acc=0, acc_list=[], path_list=[])
    # print("1",acc_l1[:3], path_l1[:3])
    # print("2",acc_l2[:3], path_l2[:3])
    acc=max(acc_l1)+max(acc_l2)
    path1=path_l1[acc_l1.index(max(acc_l1))]
    path2=path_l2[acc_l2.index(max(acc_l2))]
    # print(acc, path1, path2)
    acc_list.append(acc)
    path_=[path1,path2]
    path_list_final.append(path_)

print(len(acc_list))
print(len(path_list_final))
print(max(acc_list))
print(path_list_final[acc_list.index(max(acc_list))])



# max_release = []
# max_path = []
# dict1={}
# dict2={}
# acc_list1 = {}
# acc_list2 = {}
# path_list1 = {}
# path_list2 = {}
# for i in range(len(group1)): #loop throught all different combinations
#     dict1[i] = {"AA":valves["AA"]}
#     dict2[i]= {"AA":valves["AA"]}
#     #group1[i], group2[i]
#     for j in group1[i]:
#         dict1[i][j] = valves[j]
#     for j in group2[i]:
#         dict2[i][j] = valves[j]
#     acc_list1[i], path_list1[i] = find_all_paths2(dict1[i], start, 26)
#     acc_list2[i], path_list2[i] = find_all_paths2(dict2[i], start, 26)

#     print("path_list1:",path_list1[i])

#     print("path_list2:",path_list2[i])
#     max_release.append(max(acc_list1[i])+max(acc_list2[i]))
#     path1=path_list1[i][acc_list1[i].index(max(acc_list1[i]))]
#     path2=path_list2[i][acc_list2[i].index(max(acc_list2[i]))]
#     max_path.append([path1,path2])
    
   

# print("max_release:",max(max_release))
# print(max_path[max_release.index(max(max_release))])


group1_list: 32766
group2_list: 32766
group2_0 [['AA', 'HO'], ['AA', 'FT'], ['AA', 'HO', 'FT']]
group2_0 [['AA', 'FT', 'TJ', 'DV', 'WI', 'JG', 'IQ', 'EV', 'KS', 'RD', 'ZV', 'HS', 'JH', 'EG', 'WJ'], ['AA', 'HO', 'TJ', 'DV', 'WI', 'JG', 'IQ', 'EV', 'KS', 'RD', 'ZV', 'HS', 'JH', 'EG', 'WJ'], ['AA', 'TJ', 'DV', 'WI', 'JG', 'IQ', 'EV', 'KS', 'RD', 'ZV', 'HS', 'JH', 'EG', 'WJ']]
32768
32766
32766
1933
[['AA', 'FT', 'JH', 'DV', 'KS', 'WI', 'EV'], ['AA', 'HS', 'EG', 'TJ', 'HO']]


In [1]:
#Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
#dict{AA:[0,status,[DD,II,BB]]}
#create a dictionary of valves and their flow rates and tunnels

#or
#class valve: 
#self.flow_rate
#self.status: 0 for closed , 1 for open
#self.nexttunnels = [DD,II,BB]
"""
import re


#make a function to parse the string
def parse_string(string):
    #split ether "Valve " or " has flow rate=" or "; tunnels lead to valves " or "tunnel leads to valve"
    match = re.search(r'tunnels lead to valves', string)
    if match:
        string = re.split("Valve | has flow rate=|; tunnels lead to valves ", string)
    else:
        string = re.split("Valve | has flow rate=|; tunnel leads to valve ", string)
    string = [i for i in string if i]
    valve=string[0]
    flow_rate=int(string[1])
    adj=string[2].split(", ")
    return valve, flow_rate, adj 

print(parse_string("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB"))


class Valve:
    def __init__(self,flow_rate, adj:list, status=0):
        self.flow_rate = flow_rate
        self.status = status
        self.adj = adj

    # def update_status(self,valve_dict, time):
    #     if self.status == 0:


# def update_flow(current_flow,added_flow): 
#     current_flow+=added_flow        
#     return current_flow



with open('/Users/yang/Desktop/adventofcode/d16_valve.txt') as file:
    lines=file.read().splitlines()

# myfile=open("/Users/yang/Desktop/adventofcode/d16_valve.txt",'r')
# lines=myfile.readlines()
#parse string
#make a dictionary of valves and their flow rates and tunnels
valve_dict={}

for i in range(len(lines)):
    parsed=parse_string(lines[i])
    print(parsed)
    valve_dict[parsed[0]]=Valve(parsed[1],parsed[2])

# for i in valve_dict:
#     print("\n",i,"flow rate:",valve_dict[i].flow_rate)
#     print("status:",valve_dict[i].status)
#     print("adj:",valve_dict[i].adj)
"""

('AA', 0, ['DD', 'II', 'BB'])
('AA', 0, ['DD', 'II', 'BB'])
('BB', 13, ['CC', 'AA'])
('CC', 2, ['DD', 'BB'])
('DD', 20, ['CC', 'AA', 'EE'])
('EE', 3, ['FF', 'DD'])
('FF', 0, ['EE', 'GG'])
('GG', 0, ['FF', 'HH'])
('HH', 22, ['GG'])
('II', 0, ['AA', 'JJ'])
('JJ', 21, ['II'])


In [ ]:
#floyd warshall algorithm


In [4]:
testlist=["AA","BB","CC"]
testlist.index("AA")

0

In [2]:
#brute force
#not working because dictionary is passed by reference
"""
current_pos="AA"
time=30
acc_list=[]
current_flow=0

#dictionary is passed by reference!!!!!!!!!!!!! each branch of recursion should have its own copy of dictionary
#can stop recursion when all valves are open

def walk_through_tunnels(valve_dict, current_pos, time, current_flow, acc):
    time-=1
    acc+=current_flow
    
    if time>0:
        next_tunnels=valve_dict[current_pos].adj
        #check if status is open
        if valve_dict[current_pos].status==1:
            for i in next_tunnels:
                current_pos = i
                walk_through_tunnels(valve_dict, current_pos, time, current_flow, acc) 
        else:
            #to open the valve or not
            for k in range(2):
                if k==0:
                    valve_dict[current_pos].status=1
                    current_flow+=valve_dict[current_pos].flow_rate  #dont know why this line is not working
                    print("update flow:",current_flow)
                    # for key in valve_dict:
                    #     print(key, valve_dict[key].status)
                    walk_through_tunnels(valve_dict, current_pos, time, current_flow, acc)
                else:
                    for i in next_tunnels:
                        current_pos = i
                        walk_through_tunnels(valve_dict, current_pos, time, current_flow, acc)
    else:
        acc_list.append(acc)
        #print everyhing 100 times
        if len(acc_list)%100==0:
            print(len(acc_list))
            print(acc_list[1],acc_list[-1])
            print("current_flow:",current_flow)
            for key in valve_dict:
                print(key, valve_dict[key].status)
    



walk_through_tunnels(valve_dict, current_pos, 5, 0, 0)
acc_list=sorted(acc_list)
print(len(acc_list))
print(acc_list[1],acc_list[-1])
"""


update flow: 0
update flow: 20
update flow: 22
update flow: 23
update flow: 0
update flow: 21
update flow: 13
update flow: 0
65
0 43


In [ ]:

# while time>0:
#     next_tunnels=valve_dict[current_pos].adj
#     #check if status is open
#     if valve_dict[current_pos].status==1:
#         for i in next_tunnels:
#             current_pos = i
#             time-=1
#             #recursion
#     else:
#         #to open the valve or not
#         for k in range(2):
#             if k==0:
#                 valve_dict[current_pos].status=1
#                 acc+=update_status(valve_dict)
#                 time-=1
#             else:
#                 for i in next_tunnels:
#                     current_pos = i
#                     time-=1
#                     #recursion

# make a recursive function to calculate the acc after 30 seconds

    


In [52]:
regex1 = r"Valve\s"
regex2 = r"\shas flow rate="
regex3 = r";\stunnel\w* lead\w* to valve\w*\s"
print("test:",re.split(regex3, "Valve AA has flow rate=0; tunnels lead to valves DD, II, BB"))
print("test:",re.split(regex3, "Valve HH has flow rate=22; tunnel leads to valve GG"))

test: ['Valve AA has flow rate=0', 'DD, II, BB']
test: ['Valve HH has flow rate=22', 'GG']
